In [1]:
!pip install -q fastapi uvicorn langchain>=1.0.4 langgraph langchain-google-genai chromadb mlflow pydantic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [66]:
import os

# ---- GCP / Gemini Config ----
PROJECT_ID = os.getenv("GCP_PROJECT_ID", "your-project-id")
REGION = "us-central1"
GEMINI_MODEL = "gemini-2.0-flash"

# ---- Local Paths (for testing) ----
SQLITE_DB_PATH = "LoanDB_BlueLoans4all.sqlite"     # place your sqlite db here
CHROMA_PATH = "chroma_store"   # pre-fed chroma directory

print("Config loaded")


Config loaded


In [ ]:
import mlflow

mlflow.set_tracking_uri("http://20.75.92.162:5000/")
mlflow.set_experiment("Sid_test_loan_navigator")


<Experiment: artifact_location='mlflow-artifacts:/548246247213487410', creation_time=1765953380858, experiment_id='548246247213487410', last_update_time=1765953380858, lifecycle_stage='active', name='Sid_test_loan_navigator', tags={}>

In [68]:
from pydantic import BaseModel
from typing import Optional, Dict, Any

class LoanAgentState(BaseModel):
    user_query: str
    intent: Optional[str] = None

    sql_result: Optional[Dict[str, Any]] = None
    policy_result: Optional[str] = None
    calc_result: Optional[Dict[str, Any]] = None

    final_answer: Optional[str] = None
    errors: Optional[str] = None


In [69]:
import sqlite3

def run_sql(query: str):
    conn = sqlite3.connect(SQLITE_DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    conn.close()

    if not rows:
        return None
    return dict(rows[0])

In [70]:

from langchain.chat_models import init_chat_model

llm = init_chat_model(
    GEMINI_MODEL,
    model_provider="google_genai"
)


In [71]:
def supervisor_router(state: LoanAgentState):
    query = state.user_query.lower()

    if "prepay" in query or "what if" in query:
        decision = "sql_then_calculator"
    elif "eligible" in query or "topup" in query:
        decision = "sql_then_policy"
    elif any(k in query for k in ["emi", "amount", "due", "status"]):
        decision = "sql"
    elif any(k in query for k in ["policy", "rbi", "rules"]):
        decision = "policy"
    else:
        decision = "sql"

    mlflow.log_param("route_decision", decision)
    return decision


In [72]:

from langchain_core.prompts import ChatPromptTemplate


SQL_PROMPT = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a fintech SQL generator for a loan support system.\n"
        "CRITICAL RULES:\n"
        "1. Output ONLY a single SQL SELECT statement.\n"
        "2. Do NOT add explanations, comments, or markdown.\n"
        "3. Use ONLY this table: loan_data\n"
        "4. Allowed columns:\n"
        "   loan_id, customer_id, loan_amount, interest_rate,\n"
        "   start_date, tenure_months, monthly_emi, amount_paid,\n"
        "   next_due_date, status, topup_eligible, prepayment_limit\n"
        "5. NEVER invent table or column names.\n"
        "6. If the question cannot be answered, output exactly:\n"
        "   SELECT NULL;\n"
    ),
    ("human", "{question}")
])




In [73]:
import re

ALLOWED_TABLES = {"loan_data"}
ALLOWED_COLUMNS = {
    "loan_id", "customer_id", "loan_amount", "interest_rate",
    "start_date", "tenure_months", "monthly_emi", "amount_paid",
    "next_due_date", "status", "topup_eligible", "prepayment_limit"
}

def extract_sql(text: str) -> str:
    text = text.strip()
    text = re.sub(r"```sql|```", "", text, flags=re.IGNORECASE).strip()
    match = re.search(r"(SELECT\s.+)", text, re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else ""

def sql_agent(state: LoanAgentState):
    chain = SQL_PROMPT | llm
    raw_output = chain.invoke({"question": state.user_query}).content

    

    sql_query = extract_sql(raw_output)

    mlflow.log_text(sql_query, "generated_sql.sql")

    if not sql_query.lower().startswith("select"):
        mlflow.log_param("sql_error", "unsafe_sql")
        return {"errors": f"Unsafe SQL output: {raw_output}"}

    # Table guard
    if not any(t in sql_query.lower() for t in ALLOWED_TABLES):
        return {"errors": "Unauthorized table referenced"}

    # Block destructive ops
    if any(k in sql_query.lower() for k in ["insert", "update", "delete", "drop"]):
        return {"errors": "Write operations are not allowed"}

    result = run_sql(sql_query)
    return {"sql_result": result}


In [74]:

from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

def policy_agent(state: LoanAgentState):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectordb = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )

    docs = vectordb.similarity_search(state.user_query, k=3)
    if not docs:
        return {"errors": "No relevant policy found"}

    context = "\n".join([d.page_content for d in docs])

    PROMPT = ChatPromptTemplate.from_messages([
        ("system", "Answer strictly using policy context. Cite when possible."),
        ("human", "Question: {question}\nContext:\n{context}")
    ])

    chain = PROMPT | llm
    answer = chain.invoke({
        "question": state.user_query,
        "context": context
    }).content

    mlflow.log_text(answer, "policy_answer.txt")
    

    return {"policy_result": answer}


In [75]:
def calculate_emi(P, r, n):
    r = r / (12 * 100)
    return (P * r * (1 + r)**n) / ((1 + r)**n - 1)

# def calculator_agent(state: LoanAgentState):
#     data = state.sql_result

#     if not data:
#         return {"errors": "Loan data not found"}

#     return {
#         "calc_result": {
#             "monthly_emi": data.get("monthly_emi"),
#             "prepayment_limit": data.get("prepayment_limit"),
#             "topup_eligible": bool(data.get("topup_eligible"))
#         }
#     }

def calculator_agent(state: LoanAgentState):
    data = state.sql_result
    if not data:
        mlflow.log_param("calc_error", "missing_sql_data")
        return {"errors": "Loan data missing for calculation"}

    result = {
        "monthly_emi": data.get("monthly_emi"),
        "prepayment_limit": data.get("prepayment_limit"),
        "topup_eligible": bool(data.get("topup_eligible"))
    }

    mlflow.log_dict(result, "calculator_result.json")
    return {"calc_result": result}



In [ ]:
def response_synthesizer(state: LoanAgentState):
    if state.errors:
        final = f"Unable to process your request: {state.errors}"
    elif state.calc_result:
        final = (
            f"Your monthly EMI is ₹{state.calc_result['monthly_emi']}. "
            f"Prepayment limit is ₹{state.calc_result['prepayment_limit']}. "
            f"Top-up eligible: {'Yes' if state.calc_result['topup_eligible'] else 'No'}."
        )
    elif state.sql_result:
        final = f"Here are the loan details you requested: {state.sql_result}"
    elif state.policy_result:
        final = state.policy_result
    else:
        final = "I could not determine an answer. Please rephrase your question."

    mlflow.log_text(final, "final_answer.txt")
    return {"final_answer": final}


In [77]:
def after_sql_router(state: LoanAgentState):
    if state.errors:
        return "synthesize"

    q = state.user_query.lower()
    if "prepay" in q or "what if" in q:
        return "calculator"
    if "eligible" in q or "topup" in q:
        return "policy"

    return "synthesize"


In [78]:
from langgraph.graph import StateGraph, START

graph = StateGraph(LoanAgentState)

graph.add_node("sql", sql_agent)
graph.add_node("calculator", calculator_agent)
graph.add_node("policy", policy_agent)
graph.add_node("synthesize", response_synthesizer)

# ---- Supervisor Entry ----
graph.add_conditional_edges(
    START,
    supervisor_router,
    {
        "sql": "sql",
        "policy": "policy",
        "sql_then_calculator": "sql",
        "sql_then_policy": "sql",
    }
)

# ---- After SQL ----
graph.add_conditional_edges(
    "sql",
    after_sql_router,
    {
        "calculator": "calculator",
        "policy": "policy",
        "synthesize": "synthesize",
    }
)

# ---- Terminal ----
graph.add_edge("calculator", "synthesize")
graph.add_edge("policy", "synthesize")

loan_graph = graph.compile()


In [79]:
with mlflow.start_run(run_name="loan_navigator_query"):
    state = LoanAgentState(user_query="What is my EMI fo rloan id 2001?")
    result = loan_graph.invoke(state.dict())

result


/tmp/ipykernel_3622/551011955.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result = loan_graph.invoke(state.dict())


🏃 View run loan_navigator_query at: http://20.75.92.162:5000/#/experiments/548246247213487410/runs/7f1abb2f668247f0bc82406a8d01d6dc
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/548246247213487410


{'user_query': 'What is my EMI fo rloan id 2001?',
 'intent': None,
 'sql_result': {'monthly_emi': 3084.06},
 'policy_result': None,
 'calc_result': None,
 'final_answer': "Here are the loan details you requested: {'monthly_emi': 3084.06}",
 'errors': None}

In [57]:
def classify_intent(query: str):
    q = query.lower()
    if "policy" in q or "rbi" in q:
        return "policy"
    if "prepay" in q or "what if" in q:
        return "calculator"
    return "sql"

In [ ]:

from langgraph.graph import StateGraph, START

graph = StateGraph(LoanAgentState)

graph.add_node("sql", sql_agent)
graph.add_node("policy", policy_agent)
graph.add_node("calculator", calculator_agent)

def route(state: LoanAgentState):
    return classify_intent(state.user_query)

# ✅ USE START (not "start")
graph.add_conditional_edges(
    START,
    route,
    {
        "sql": "sql",
        "policy": "policy",
        "calculator": "calculator"
    }
)

loan_graph = graph.compile()



In [63]:

query = "WHat re the RBI rules for prepayment"
state = LoanAgentState(user_query=query)
result = loan_graph.invoke(state.dict())
result

/tmp/ipykernel_3622/1147035642.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result = loan_graph.invoke(state.dict())


{'user_query': 'WHat re the RBI rules for prepayment',
 'intent': None,
 'sql_result': None,
 'policy_result': 'BlueLoans4all follows all RBI norms for NBFCs as outlined in Master Circular DNBR.PD.007/03.10.119/2016-17 (Page 3). Customer consent must be taken for pre-closure (Page 3).',
 'calc_result': None,
 'final_answer': None,
 'errors': None}

In [60]:
import sqlite3

conn = sqlite3.connect("LoanDB_BlueLoans4all.sqlite")
cur = conn.cursor()

cur.execute("SELECT loan_id, monthly_emi FROM loan_data LIMIT 10;")
cur.fetchall()


[(2001, 3084.06),
 (2002, 2686.56),
 (2003, 1405.21),
 (2004, 4573.65),
 (2005, 2353.67),
 (2006, 3583.28),
 (2007, 3600.97),
 (2008, 4215.63),
 (2009, 1984.92),
 (2010, 3014.38)]

In [61]:
cur.execute("SELECT * FROM loan_data WHERE loan_id = 101;")
cur.fetchall()


[]

In [47]:
import sqlite3

conn = sqlite3.connect("LoanDB_BlueLoans4all.sqlite")
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

conn.close()
tables


[('loan_data',)]

In [48]:
conn = sqlite3.connect("LoanDB_BlueLoans4all.sqlite")
cur = conn.cursor()

cur.execute("PRAGMA table_info(loan_data);")
cur.fetchall()


[(0, 'loan_id', 'INTEGER', 0, None, 0),
 (1, 'customer_id', 'INTEGER', 0, None, 0),
 (2, 'loan_amount', 'REAL', 0, None, 0),
 (3, 'interest_rate', 'REAL', 0, None, 0),
 (4, 'start_date', 'DATE', 0, None, 0),
 (5, 'tenure_months', 'INTEGER', 0, None, 0),
 (6, 'monthly_emi', 'REAL', 0, None, 0),
 (7, 'amount_paid', 'REAL', 0, None, 0),
 (8, 'next_due_date', 'TEXT', 0, None, 0),
 (9, 'status', 'TEXT', 0, None, 0),
 (10, 'topup_eligible', 'INTEGER', 0, None, 0),
 (11, 'prepayment_limit', 'REAL', 0, None, 0)]

In [45]:
import os

DB_PATH = "LoanDB_BlueLoans4all.sqlite"

print("Current working directory:", os.getcwd())
print("DB exists:", os.path.exists(DB_PATH))
print("File size (bytes):", os.path.getsize(DB_PATH) if os.path.exists(DB_PATH) else "NA")



Current working directory: /home/zadmin/Desktop/B6_GAAP_GCP/Capstone
DB exists: True
File size (bytes): 81920


## Chroma db generation

In [20]:

!pip install -q langchain>=1.0.4 langchain-google-genai chromadb pypdf



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [22]:

import os

from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [23]:
POLICY_DOCS_PATH = "docs"
CHROMA_PATH = "chroma_store"

os.makedirs(CHROMA_PATH, exist_ok=True)

print("Policy docs path:", POLICY_DOCS_PATH)
print("Chroma output path:", CHROMA_PATH)

Policy docs path: docs
Chroma output path: chroma_store


In [24]:

documents = []

for file in os.listdir(POLICY_DOCS_PATH):
    full_path = os.path.join(POLICY_DOCS_PATH, file)

    if file.endswith(".pdf"):
        loader = PyPDFLoader(full_path)
        documents.extend(loader.load())

    elif file.endswith(".txt"):
        loader = TextLoader(full_path)
        documents.extend(loader.load())

print(f"Loaded {len(documents)} document pages")


Loaded 20 document pages


In [25]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")


Created 40 chunks


In [26]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

In [28]:

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=CHROMA_PATH
)


print("✅ Chroma DB created successfully")


✅ Chroma DB created successfully
